# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

<ipython-input-1-1499ab5789c7>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
sys.path.append('../')
import common as com
import pytorch_modeler as modeler
from pytorch_model import ResNet38 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/CL-VAE_experiments/output/CL-AD_MEAN/config.yaml'

In [7]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [8]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    
    # dev_source valid
    dev_test_paths = [f"{dev_dir}/{machine_type}/test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/test")]
    dev_test_paths = sorted(dev_test_paths)
    
    # dev_target valid
    #dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    #dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['dev_test'] = dev_test_paths

In [9]:
#dev_paths['fan']['train'][-1]

    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    logger.info('TRAINING')
    # parameter setting
    in_features = ext_data['train']['features'].shape[1]
    mid_size = config['param']['mid_size']
    latent_size = config['param']['latent_size']
    id_s = com.get_id(ext_data['train']['wav_names'])
    num_classes = len(np.unique(id_s))
    print(np.unique(id_s))
    net = Model(in_features, mid_size, latent_size, num_classes)
    optimizer = optim.Adam(net.parameters(), lr=1e-4)
    num_epochs = config['param']['num_epochs']
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e-2, 
    #                                          max_lr=1e-3, epochs=num_epochs, steps_per_epoch=len(dataloaders_dict['train']))
    # training
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path)
    
    com.toc()

## training

In [10]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    # out path
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    score_out_path = OUT_SCORE_DIR + '/{}_score.csv'.format(machine_type)
    pred_out_path = OUT_PRED_DIR + '/{}_pred.csv'.format(machine_type)
    # parameter setting
    logger.info('training')
    id_s = com.get_id(dev_paths[machine_type]['train'])
    num_classes = len(np.unique(id_s))
    print(np.unique(id_s))
    net = Model(sample_rate=config['param']['sample_rate'],
                window_size=config['param']['window_size'],
                hop_size=config['param']['hop_size'],
                mel_bins=config['param']['mel_bins'],
                fmin=config['param']['fmin'],
                fmax=config['param']['fmax'],
                num_classes=num_classes)
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_ROOT'])
    net.load_state_dict(pretrained_dict['model'], strict=False)
    #for param in net.parameters():
    #    param.requires_grad = False
    #print(list(net.children()))
    #grad_layers = list(net.children())[-1]
    #print(grad_layers)
    #for grad_layer in grad_layers:
    #for param in grad_layers.parameters():
    #    param.requires_grad = True
    optimizer = optim.Adam(net.parameters(), lr=1e-3)
    num_epochs = config['param']['num_epochs']
    output_dicts = modeler.train_net(net, dataloaders_dict, optimizer, num_epochs, writer, model_out_path, score_out_path, pred_out_path, num_classes)
    
    #out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    #pd.to_pickle(output_dicts, out_path)
    #logger.info(f'SAVE SUCCESS : {out_path}')
    
    com.toc()

In [11]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

# run

In [12]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-07-10 23:56:18,173 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2021-07-10 23:56:18,174 - 00_train.py - INFO - MAKE DATA_LOADER
2021-07-10 23:56:19,004 - 00_train.py - INFO - training


[0 1 2 3 4 5 6]


  0%|          | 0/26 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 8/8 [00:03<00:00,  2.48it/s]
2021-07-10 23:56:49,832 - pytorch_modeler.py - INFO - epoch:1/50, tr_loss:5710.515531, val_loss:5687.921265, val_AUC_hmean:0.508875, val_pAUC_hmean:0.504258,


...

100%|██████████| 4/4 [00:01<00:00,  2.06it/s]
2021-07-11 01:56:53,824 - pytorch_modeler.py - INFO - epoch:38/50, tr_loss:3684.170070, val_loss:4378.906250, val_AUC_hmean:0.880520, val_pAUC_hmean:0.739946,
100%|██████████| 4/4 [00:01<00:00,  2.08it/s]
2021-07-11 01:57:14,631 - pytorch_modeler.py - INFO - epoch:39/50, tr_loss:3679.327838, val_loss:4301.536621, val_AUC_hmean:0.880052, val_pAUC_hmean:0.739946,
100%|██████████| 4/4 [00:01<00:00,  2.07it/s]
2021-07-11 01:57:35,466 - pytorch_modeler.py - INFO - epoch:40/50, tr_loss:3674.830588, val_loss:4298.793396, val_AUC_hmean:0.876634, val_pAUC_hmean:0.739946,
100%|██████████| 4/4 [00:02<00:00,  1.92it/s]
2021-07-11 01:57:56,607 - pytorch_modeler.py - INFO - epoch:41/50, tr_loss:3671.566427, val_loss:4361.606934, val_AUC_hmean:0.880520, val_pAUC_hmean:0.739946,
100%|██████████| 4/4 [00:01<00:00,  2.08it/s]
2021-07-11 01:58:17,486 - pytorch_modeler.py - INFO - epoch:42/50, tr_loss:3668.650072, val_loss:4340.982300, val_AUC_hmean:0.880520, 

,AUC,pAUC
id_0,0.915378,0.667846
id_2,1.000000,1.000000
id_4,0.992833,0.962281
id_6,0.980083,0.895175
mean,0.972074,0.881326
h_mean,0.970877,0.859541


100%|██████████| 4/4 [00:01<00:00,  2.05it/s]
2021-07-11 02:00:44,422 - pytorch_modeler.py - INFO - epoch:49/50, tr_loss:3715.699548, val_loss:4536.636963, val_AUC_hmean:0.337077, val_pAUC_hmean:0.585492,
100%|██████████| 4/4 [00:01<00:00,  2.05it/s]
2021-07-11 02:01:05,309 - pytorch_modeler.py - INFO - epoch:50/50, tr_loss:3711.946608, val_loss:5166.386841, val_AUC_hmean:0.062238, val_pAUC_hmean:0.533233,
2021-07-11 02:01:05,310 - pytorch_modeler.py - INFO - best model, epoch:48/50, train_losses:3715.672098, val_pAUC_hmean:0.859541,


,AUC,pAUC
id_0,0.915378,0.667846
id_2,1.000000,1.000000
id_4,0.992833,0.962281
id_6,0.980083,0.895175
mean,0.972074,0.881326
h_mean,0.970877,0.859541


elapsed time: 1129.574299097 [sec]


In [13]:
#run(machine_types[4], dev_paths)